In [1]:
using Plots, LinearAlgebra, DifferentialEquations
plotlyjs()

#gr()

WebIO._IJuliaInit()

Plots.PlotlyJSBackend()

In [2]:
include("C:/Users/mjski/Desktop/Active-Work/git-projects/scripts/scripts/spacecraft_adcs.jl")
include("C:/Users/mjski/Desktop/Active-Work/git-projects/scripts/scripts/orbital_dynamics.jl")
include("C:/Users/mjski/Desktop/Active-Work/git-projects/scripts/scripts/orbital_elements.jl")
include("C:/Users/mjski/Desktop/Active-Work/git-projects/scripts/scripts/gnc.jl")

get_lunar_ephemeris (generic function with 1 method)

# Defining Functions

In [3]:
function diffeq_nonimpulsive_orbital_maneuver2(initial_conditions, time_span, params; solver_args...)
    function differential_system!(du, u, p, t)
        x, y, z, ẋ, ẏ, ż, m = u
        μ, Isp, Tx_func, Ty_func, Tz_func = p

        Tx = Tx_func(u, p, t)
        Ty = Ty_func(u, p, t)
        Tz = Tz_func(u, p, t)
        r = sqrt(x^2 + y^2 + z^2)
        v = sqrt(ẋ^2 + ẏ^2 + ż^2)
        
        du[1] = ẋ
        du[2] = ẏ
        du[3] = ż
        du[4] = -μ*x / r^3 + (Tx / m) * (ẋ / v)
        du[5] = -μ*y / r^3 + (Ty / m) * (ẏ / v)
        du[6] = -μ*z / r^3 + (Tz / m) * (ż / v)
        du[7] = -1e3 * sqrt(Tx^2 + Ty^2 + Tz^2) / (Isp * 9.80665)
    end

    problem = ODEProblem(differential_system!, initial_conditions, time_span, params)
    solution = solve(problem; solver_args...)

    return solution
end

diffeq_nonimpulsive_orbital_maneuver2 (generic function with 1 method)

In [4]:
function diffeq_two_body_simple(initial_conditions, time_span, params; solver_args...)
    function differential_system!(du, u, p, t)
        x, y, z, vx, vy, vz = u
        μ, mass = p

        r = sqrt(x^2 + y^2 + z^2)

        du[1] = vx
        du[2] = vy
        du[3] = vz
        du[4] = -μ * x / r^3
        du[5] = -μ * y / r^3
        du[6] = -μ * z / r^3
    end

    problem = ODEProblem(differential_system!, initial_conditions, time_span, params)
    solution = solve(problem; solver_args...)

    return solution
end

diffeq_two_body_simple (generic function with 1 method)

In [5]:
function get_trajectory_results(ODE_solution, times)
    solution_interp = ODE_solution(times)
    
    x = solution_interp[1,1:end]
    y = solution_interp[2,1:end]
    z = solution_interp[3,1:end]

    vx = solution_interp[4,1:end]
    vy = solution_interp[5,1:end]
    vz = solution_interp[6,1:end]
    
    return solution_interp, times, x, y, z, vx, vy, vz
end

get_trajectory_results (generic function with 1 method)

In [6]:
function get_burn_results(ODE_solution, times)
    solution_interp = ODE_solution(times)
    
    x = solution_interp[1,1:end]
    y = solution_interp[2,1:end]
    z = solution_interp[3,1:end]

    vx = solution_interp[4,1:end]
    vy = solution_interp[5,1:end]
    vz = solution_interp[6,1:end]
    
    m = solution_interp[7,1:end]
    
    return solution_interp, times, x, y, z, vx, vy, vz, m
end

get_burn_results (generic function with 1 method)

# Trajectory 1

Initial trajectory after deployment from launcher, 300x350 km orbit starting at periapsis

In [7]:
re = 6378
μ = 398600

rp_init = re + 300
ra_init = re + 350

# setting up spacecraft parameters
total_thrust = 0.200 #kN
curie_isp = 320
kickstage_wet_mass = 100
kickstage_dry_mass = 30
prop_mass = kickstage_wet_mass - kickstage_dry_mass
payload_mass = 160

total_mass = kickstage_wet_mass + payload_mass

e_init = (ra_init - rp_init) / (ra_init + rp_init)
i_init = deg2rad(55)
Ω_init = deg2rad(-45)
ω_init = 0
θ_init = 0

h_init = sqrt(rp_init * μ * (1 + e_init * cos(θ_init)))

# e, i, asc, arg, theta = orbital_elements
r_init, v_init = geocentric_state_vector_transform(h_init, [e_init, i_init, Ω_init, ω_init, θ_init]);

In [8]:
init_traj1 = [r_init[1], r_init[2], r_init[3], v_init[1], v_init[2], v_init[3]]
params_traj1 = [μ, total_mass]
tspan_traj1 = (0.0, 0.5*(((rp_init + ra_init)/2)^(3/2))*2*π/sqrt(μ))

sol_traj1 = diffeq_two_body_simple(init_traj1, tspan_traj1, params_traj1, solver=Vern9(),reltol=1e-12, abstol=1e-12)

traj1_interp, times_traj1, x_traj1, y_traj1, z_traj1, vx_traj1, vy_traj1, vz_traj1 = get_trajectory_results(sol_traj1, LinRange(0.0, tspan_traj1[2], 500));

In [9]:
plot(times_traj1, sqrt.(x_traj1.^2 .+ y_traj1.^2 .+ z_traj1.^2) .- re, label="")

# Burn 1

First circularization burn, circularizing at 350 km for first satellite deployment

In [10]:
burn1_time = 27.25

# thrust functions
# assuming thrust pointed constantly at prograde direction
function Tx_func_burn1(u, p, t)
    x, y, z, ẋ, ẏ, ż, m = u
    μ, Isp, Tx_func, Ty_func, Tz_func = p
    
    v = sqrt(ẋ^2 + ẏ^2 + ż^2)
    xvel_unit = ẋ / v
    
    t < burn1_time && m - (payload_mass + kickstage_dry_mass) <= prop_mass ? thrust_x = -xvel_unit * total_thrust : thrust_x = 0
    
    return thrust_x
end

function Ty_func_burn1(u, p, t)
    x, y, z, ẋ, ẏ, ż, m = u
    μ, Isp, Tx_func, Ty_func, Tz_func = p
    
    v = sqrt(ẋ^2 + ẏ^2 + ż^2)
    yvel_unit = ẏ / v
    
    t < burn1_time && m - (payload_mass + kickstage_dry_mass) <= prop_mass ? thrust_y = -yvel_unit * total_thrust : thrust_y = 0
    
    return thrust_y
end

function Tz_func_burn1(u, p, t)
    x, y, z, ẋ, ẏ, ż, m = u
    μ, Isp, Tx_func, Ty_func, Tz_func = p
    
    v = sqrt(ẋ^2 + ẏ^2 + ż^2)
    zvel_unit = ż / v
    
    t < burn1_time && m - (payload_mass + kickstage_dry_mass) <= prop_mass ? thrust_z = -zvel_unit * total_thrust : thrust_z = 0

    return thrust_z
end

init_burn1 = [x_traj1[end], y_traj1[end], z_traj1[end], vx_traj1[end], vy_traj1[end], vz_traj1[end], total_mass]
params_burn1 = [μ, curie_isp, Tx_func_burn1, Ty_func_burn1, Tz_func_burn1]
tspan_burn1 = (0.0, burn1_time)

sol_burn1 = diffeq_nonimpulsive_orbital_maneuver2(init_burn1, tspan_burn1, params_burn1, solver=Vern9(),reltol=1e-12, abstol=1e-12)

burn1_interp, times_burn1, x_burn1, y_burn1, z_burn1, vx_burn1, vy_burn1, vz_burn1, m_burn1 = get_burn_results(sol_burn1, LinRange(0.0, burn1_time, 500))
m_burn1_final = m_burn1[end];

# Trajectory 2

Initial coasting trajectory at 350 km circular orbit, five minutes for payload deployment

In [11]:
init_traj2 = [x_burn1[end], y_burn1[end], z_burn1[end], vx_burn1[end], vy_burn1[end], vz_burn1[end]]
params_traj2 = [μ, m_burn1_final]

# assume 5 minutes for payload deployment
tspan_traj2 = (0.0, 5*60.0)

sol_traj2 = diffeq_two_body_simple(init_traj2, tspan_traj2, params_traj2, solver=Vern9(),reltol=1e-12, abstol=1e-12)

traj2_interp, times_traj2, x_traj2, y_traj2, z_traj2, vx_traj2, vy_traj2, vz_traj2 = get_trajectory_results(sol_traj2, LinRange(0.0, tspan_traj2[2], 500));

In [12]:
alt_traj2 = sqrt.(x_traj2.^2 .+ y_traj2.^2 .+ z_traj2.^2) .- re

plot(times_burn1, sqrt.(x_burn1.^2 .+ y_burn1.^2 .+ z_burn1.^2) .- re, label="Burn 1")
plot!(times_traj2 .+ times_burn1[end], alt_traj2, label="Trajectory 2")

In [13]:
println("MSE: $(sum((alt_traj2 .- 350).^2) / length(alt_traj2))\nMass: $m_burn1_final")

MSE: 0.6653633860691461
Mass: 260.0


# Burn 2

Burn from circular 350 km orbit to 350x450 km elliptic orbit, transfer burn to 450 km circular orbit

In [13]:
burn2_time = 35.75

# thrust functions
# assuming thrust pointed constantly at prograde direction
function Tx_func_burn2(u, p, t)
    x, y, z, ẋ, ẏ, ż, m = u
    μ, Isp, Tx_func, Ty_func, Tz_func = p
    
    v = sqrt(ẋ^2 + ẏ^2 + ż^2)
    xvel_unit = ẋ / v
    
    t < burn2_time && m - (payload_mass/2 + kickstage_dry_mass) <= prop_mass ? thrust_x = -xvel_unit * total_thrust : thrust_x = 0
    
    return thrust_x
end

function Ty_func_burn2(u, p, t)
    x, y, z, ẋ, ẏ, ż, m = u
    μ, Isp, Tx_func, Ty_func, Tz_func = p
    
    v = sqrt(ẋ^2 + ẏ^2 + ż^2)
    yvel_unit = ẏ / v
    
    t < burn2_time && m - (payload_mass/2 + kickstage_dry_mass) <= prop_mass ? thrust_y = -yvel_unit * total_thrust : thrust_y = 0
    
    return thrust_y
end

function Tz_func_burn2(u, p, t)
    x, y, z, ẋ, ẏ, ż, m = u
    μ, Isp, Tx_func, Ty_func, Tz_func = p
    
    v = sqrt(ẋ^2 + ẏ^2 + ż^2)
    zvel_unit = ż / v
    
    t < burn2_time && m - (payload_mass/2 + kickstage_dry_mass) <= prop_mass ? thrust_z = -zvel_unit * total_thrust : thrust_z = 0

    return thrust_z
end

init_burn2 = [x_traj2[end], y_traj2[end], z_traj2[end], vx_traj2[end], vy_traj2[end], vz_traj2[end], m_burn1_final - payload_mass/2]
params_burn2 = [μ, curie_isp, Tx_func_burn2, Ty_func_burn2, Tz_func_burn2]
tspan_burn2 = (0.0, burn2_time)

sol_burn2 = diffeq_nonimpulsive_orbital_maneuver2(init_burn2, tspan_burn2, params_burn2, solver=Vern9(),reltol=1e-12, abstol=1e-12)

burn2_interp, times_burn2, x_burn2, y_burn2, z_burn2, vx_burn2, vy_burn2, vz_burn2, m_burn2 = get_burn_results(sol_burn2, LinRange(0.0, burn2_time, 500))
m_burn2_final = m_burn2[end];

# Trajectory 3

Coasting trajectory to 450 km altitude

In [14]:
init_traj3 = [x_burn2[end], y_burn2[end], z_burn2[end], vx_burn2[end], vy_burn2[end], vz_burn2[end]]
params_traj3 = [μ, m_burn2_final]

tspan_traj3 = (0.0, 46.3*60.0)

sol_traj3 = diffeq_two_body_simple(init_traj3, tspan_traj3, params_traj3, solver=Vern9(),reltol=1e-12, abstol=1e-12)

traj3_interp, times_traj3, x_traj3, y_traj3, z_traj3, vx_traj3, vy_traj3, vz_traj3 = get_trajectory_results(sol_traj3, LinRange(0.0, tspan_traj3[2], 500));

In [15]:
alt_traj3 = sqrt.(x_traj3.^2 .+ y_traj3.^2 .+ z_traj3.^2) .- re

plot(times_burn2, sqrt.(x_burn2.^2 .+ y_burn2.^2 .+ z_burn2.^2) .- re, label="Burn 2", legend=:topleft)
plot!(times_traj3 .+ times_burn2[end], alt_traj3, label="Trajectory 3")

# Burn 3

Circularization and plane change burn at 450 km altitude, inclining orbit by 10 degrees

In [87]:
m_burn2_final - (kickstage_dry_mass + payload_mass/2)

65.984867412195

In [88]:
ṁ = 200 / (curie_isp * 9.80665)

Δm = (1 - exp(-1388.5 / (curie_isp * 9.80665))) * m_burn2_final
#1388.5
#ṁ = mass / time
burn3_time = Δm / ṁ

987.2992941730197

In [25]:
burn3_time = 987.2992941730197
burn3_angle_from_vel = deg2rad(-85)

# x̂ = R ./ norm(R)
# ŷ = V ./ norm(V)
# ẑ = cross(x̂, ŷ) ./ norm(cross(x̂, ŷ))

# return x̂, ŷ, ẑ

# thrust functions
# assuming thrust pointed constantly at prograde direction
function Tx_func_burn3(u, p, t)
    x, y, z, ẋ, ẏ, ż, m = u
    μ, Isp, Tx_func, Ty_func, Tz_func = p
    
#     v = sqrt(ẋ^2 + ẏ^2 + ż^2)
    
    LVLH_x = [x, y, z] ./ sqrt(x^2 + y^2 + z^2)
    C = cos(burn3_angle_from_vel) .+ (1 - cos(burn3_angle_from_vel)) .* LVLH_x * LVLH_x' .- sin(burn3_angle_from_vel)*cross_mat(LVLH_x)
    vel_new = C * [ẋ, ẏ, ż]
    xvel_unit = vel_new[1] / norm(vel_new)
    
    t < burn3_time && m - (payload_mass/2 + kickstage_dry_mass) <= prop_mass ? thrust_x = xvel_unit * total_thrust : thrust_x = 0
    
    return thrust_x
end

function Ty_func_burn3(u, p, t)
    x, y, z, ẋ, ẏ, ż, m = u
    μ, Isp, Tx_func, Ty_func, Tz_func = p
    
#     v = sqrt(ẋ^2 + ẏ^2 + ż^2)
    
    LVLH_x = [x, y, z] ./ sqrt(x^2 + y^2 + z^2)
    C = cos(burn3_angle_from_vel) .+ (1 - cos(burn3_angle_from_vel)) .* LVLH_x * LVLH_x' .- sin(burn3_angle_from_vel)*cross_mat(LVLH_x)
    vel_new = C * [ẋ, ẏ, ż]
    yvel_unit = vel_new[2] / norm(vel_new)
    
    t < burn3_time && m - (payload_mass/2 + kickstage_dry_mass) <= prop_mass ? thrust_y = yvel_unit * total_thrust : thrust_y = 0
    
    return thrust_y
end

function Tz_func_burn3(u, p, t)
    x, y, z, ẋ, ẏ, ż, m = u
    μ, Isp, Tx_func, Ty_func, Tz_func = p
    
#     v = sqrt(ẋ^2 + ẏ^2 + ż^2)
    
    LVLH_x = [x, y, z] ./ sqrt(x^2 + y^2 + z^2)
    C = cos(burn3_angle_from_vel) .+ (1 - cos(burn3_angle_from_vel)) .* LVLH_x * LVLH_x' .- sin(burn3_angle_from_vel)*cross_mat(LVLH_x)
    vel_new = C * [ẋ, ẏ, ż]
    zvel_unit = vel_new[3] / norm(vel_new)
    
    t < burn3_time && m - (payload_mass/2 + kickstage_dry_mass) <= prop_mass ? thrust_z = zvel_unit * total_thrust : thrust_z = 0

    return thrust_z
end

init_burn3 = [x_traj3[end], y_traj3[end], z_traj3[end], vx_traj3[end], vy_traj3[end], vz_traj3[end], m_burn2_final]
params_burn3 = [μ, curie_isp, Tx_func_burn3, Ty_func_burn3, Tz_func_burn3]
tspan_burn3 = (0.0, burn3_time)

sol_burn3 = diffeq_nonimpulsive_orbital_maneuver2(init_burn3, tspan_burn3, params_burn3, solver=Vern9(),reltol=1e-12, abstol=1e-12)

burn3_interp, times_burn3, x_burn3, y_burn3, z_burn3, vx_burn3, vy_burn3, vz_burn3, m_burn3 = get_burn_results(sol_burn3, LinRange(0.0, burn3_time, 500))
m_burn3_final = m_burn3[end];

In [26]:
m_burn3_final - (kickstage_dry_mass + payload_mass/2)

3.062048829167253

In [27]:
post_burn3_inclination = get_orbital_elements([x_burn3[end], y_burn3[end], z_burn3[end]], [vx_burn3[end], vy_burn3[end], vz_burn3[end]])[3]
initial_inclination = get_orbital_elements([x_traj3[end], y_traj3[end], z_traj3[end]], [vx_traj3[end], vy_traj3[end], vz_traj3[end]])[3]

rad2deg(initial_inclination), rad2deg(post_burn3_inclination)

(55.052563470713146, 57.15017301369578)

# Trajectory 4

In [28]:
init_traj4 = [x_burn3[end], y_burn3[end], z_burn3[end], vx_burn3[end], vy_burn3[end], vz_burn3[end]]
params_traj4 = [μ, m_burn3_final]

tspan_traj4 = (0.0, 120.0*60.0)

sol_traj4 = diffeq_two_body_simple(init_traj4, tspan_traj4, params_traj4, solver=Vern9(),reltol=1e-12, abstol=1e-12)

traj4_interp, times_traj4, x_traj4, y_traj4, z_traj4, vx_traj4, vy_traj4, vz_traj4 = get_trajectory_results(sol_traj4, LinRange(0.0, tspan_traj4[2], 500));

In [29]:
plot(times_traj3, alt_traj3, label="Trajectory 3", legend=:topleft)

plot!(times_burn3 .+ times_traj3[end], sqrt.(x_burn3.^2 .+ y_burn3.^2 .+ z_burn3.^2) .- re, label="Burn 3")

plot!(times_traj4 .+ times_traj3[end].+times_burn3[end], sqrt.(x_traj4.^2 .+ y_traj4.^2 .+ z_traj4.^2) .- re, label="Trajectory 4")

In [69]:
plot(x_traj3, y_traj3, z_traj3, size=(900,900), label="Trajectory 3")
plot!(x_burn3, y_burn3, z_burn3, label="Burn 3")
plot!(x_traj4, y_traj4, z_traj4, label="Trajectory 4")


X(r,theta,phi) = r * sin(theta) * sin(phi)
Y(r,theta,phi) = r * sin(theta) * cos(phi)
Z(r,theta,phi) = r * cos(theta)

thetas = range(0, stop=2*π, length=50)
phis   = range(0, stop=2*π, length=50)

xs = [X(re, theta, phi) for theta in thetas, phi in phis] 
ys = [Y(re, theta, phi) for theta in thetas, phi in phis]
zs = [Z(re, theta, phi) for theta in thetas, phi in phis]

surface!(xs, ys, zs, colorbar=nothing, color=:lightblue)

# Visualizations

In [20]:
total_x = vcat(x_traj1, x_burn1, x_traj2, x_burn2, x_traj3)
total_y = vcat(y_traj1, y_burn1, y_traj2, y_burn2, y_traj3)
total_z = vcat(z_traj1, z_burn1, z_traj2, z_burn2, z_traj3)

total_r = sqrt.(total_x.^2 .+ total_y.^2 .+ total_z.^2)

plot(total_r .- re, label="")

In [56]:
plot(x_burn1, y_burn1, z_burn1, label="Burn 1", size=(900,900))
plot!(x_traj1, y_traj1, z_traj1, label="Trajectory 1")

X(r,theta,phi) = r * sin(theta) * sin(phi)
Y(r,theta,phi) = r * sin(theta) * cos(phi)
Z(r,theta,phi) = r * cos(theta)

thetas = range(0, stop=2*π, length=50)
phis   = range(0, stop=2*π, length=50)

xs = [X(re, theta, phi) for theta in thetas, phi in phis] 
ys = [Y(re, theta, phi) for theta in thetas, phi in phis]
zs = [Z(re, theta, phi) for theta in thetas, phi in phis]

surface!(xs, ys, zs, colorbar=nothing, color=:lightblue)